In [ ]:
import numpy as np
import pandas as pd

from scipy.sparse import coo_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import preprocessing

In [ ]:
batch_size = 128
embedding_size = 8
epoch_count = 10
sequence_length = 170
split_ratio = 7 / 3
variety_threshold = 500
vocabulary_size = 12000
wide_size = 256

# Data

In [ ]:
data = pd.read_csv('data/data.csv')
data = data[['description', 'variety', 'price']]
data = data[~data['price'].isnull()]

In [ ]:
varieties = data['variety'].value_counts()
varieties = varieties[varieties >= variety_threshold].index
data = data[data['variety'].isin(varieties.values)]

In [ ]:
split = int(split_ratio / (1 + split_ratio) * len(data))
data = data.sample(frac=1, random_state=42)
data_train, data_test = data[:split], data[:split]

# Encoding

In [ ]:
encoder = LabelEncoder()
encoder.fit(data_train['variety'])
variety_count = len(encoder.classes_)

In [ ]:
variety_wide_train = encoder.transform(data_train['variety']).reshape([-1, 1])
variety_wide_test = encoder.transform(data_test['variety']).reshape([-1, 1])

In [ ]:
encoder = OneHotEncoder()
encoder.fit(variety_wide_train);

In [ ]:
variety_wide_train = encoder.transform(variety_wide_train)
variety_wide_test = encoder.transform(variety_wide_test)

# Tokenization

In [ ]:
vectorizer = CountVectorizer(binary=True, dtype=bool, max_features=vocabulary_size)
vectorizer.fit(data_train['description']);

In [ ]:
def convert(descriptions):
    return [
        [vectorizer.vocabulary_.get(token, 0) for token in analyzer(description)]
        for description in descriptions
    ]

analyzer = vectorizer.build_analyzer()
description_deep_train = convert(data_train['description'].values)
description_deep_train = preprocessing.sequence.pad_sequences(description_deep_train, maxlen=sequence_length)
description_deep_test = convert(data_test['description'].values)
description_deep_test = preprocessing.sequence.pad_sequences(description_deep_test, maxlen=sequence_length)

In [ ]:
description_wide_train = vectorizer.transform(data_train['description'])
description_wide_test = vectorizer.transform(data_test['description'])

# Modeling

## Deep

In [ ]:
description_input = layers.Input(shape=(sequence_length,))
embedding_layer = layers.Embedding(vocabulary_size, embedding_size, input_length=sequence_length)(description_input)
embedding_layer = layers.Flatten()(embedding_layer)
deep_output = layers.Dense(1, activation='linear')(embedding_layer)
deep_model = models.Model(inputs=description_input, outputs=deep_output)
deep_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
deep_model.summary()

## Wide

In [ ]:
description_input = layers.Input(shape=(vocabulary_size,))
variety_input = layers.Input(shape=(variety_count,))
wide_input = layers.concatenate([description_input, variety_input])
wide_layer = layers.Dense(wide_size, activation='relu')(wide_input)
wide_output = layers.Dense(1)(wide_layer)
wide_model = models.Model(inputs=[description_input, variety_input], outputs=wide_output)
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
wide_model.summary()

## Composite

In [ ]:
composite_input = layers.concatenate([deep_model.output, wide_model.output])
composite_output = layers.Dense(1)(composite_input)
composite_model = models.Model(inputs=[deep_model.input] + wide_model.input, outputs=composite_output)
composite_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
composite_model.summary()

# Training

In [ ]:
composite_model.fit([description_deep_train] + [description_wide_train, variety_wide_train],
                    data_train['price'], epochs=epoch_count, batch_size=batch_size)

# Evaluation

In [ ]:
composite_model.evaluate([description_deep_test] + [description_wide_test, variety_wide_test],
                         data_test['price'], batch_size=batch_size)